# LangChain 快速入門
[LangChain](https://github.com/langchain-ai/langchain) 是針對大型語言模型 (LLM) 應用的相關實作加以抽象化的程式語言框架，支援 Python 與 Node.js 兩種程式語言，也能夠支援包含 OpenAI GPT 等多種 LLM ，在 LangChain 中不僅僅限於 LLM 之呼叫，還能納入一系列相關工具的呼叫。藉由 LangChain 所提供的 Chain 之標準介面，目前已經有著大量工具與資料來源能夠整合至 LangChain 框架之內。鍊是一種通用的概念，可以將特定運算簡化為一個可以重複使用；包含輸入/處理/輸出的一個元件，下圖就是 LangChain 最常用的鏈 LLMChain 為例，它結合了 PromptTemplate 獲取用戶輸入，支援多種大型語言模型，並將大型語言模型輸出的內容透過 Output Parser 轉換為特定格式， Chain 也可以將輸出再傳遞給其他鍊組合出更複雜的應用。

![Chain 概念，以 LLMChain 為例](./assets/chain-concept.png)

載入會使用到的套件

In [68]:
%pip install langchain langchain-core langchain-openai langchain-community langchain-experimental python-dotenv wikipedia faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1. 運用輔助函式庫，從環境變數取得 Azure OpenAI API 相關資訊

開始使用 LangChain，Azure OpenAI Service 的主要參數都是透過環境變數設定完成，只需要指定以下幾個環境變數即可

+ AZURE_OPENAI_ENDPOINT
+ AZURE_OPENAI_API_KEY
+ AZURE_OPENAI_API_VERSION
+ CHAT_DEPLOYMENT_NAME
+ EMBEDDINS_DEPLOYMENT_NAME

In [3]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai.chat_models.azure import AzureChatOpenAI
from langchain_community.document_loaders import WikipediaLoader
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

# 載入環境變數
load_dotenv()

# 設定呼叫 OpenAI API 所需連線資訊
chat_model = os.getenv("CHAT_DEPLOYMENT_NAME")
emb_model = os.getenv("EMBEDDINS_DEPLOYMENT_NAME")
api_ver = os.getenv("AZURE_OPENAI_API_VERSION")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

## 2. 最簡單的自動完成範例


LangChain 可以透過針對 Azure OpenAI Service 所設計的 AzureChatOpenAI 來進行對話。AzureChatOpenAI 會將使用者的輸入轉換成 OpenAI  chat completions API 所需的格式與 Azure OpenAI Service 呼叫端點，並將 chat completions API 的回應轉換成 LangChain 的格式。

In [104]:
chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)
respone = chat.invoke("世界上最高的三座山?")
print(respone)

content='世界上最高的三座山按海拔高度排名如下：\n\n1. **珠穆朗玛峰（Mount Everest）**  \n   - **海拔高度**：8,848.86米（29,031.7英尺）  \n   - **位置**：位于中国与尼泊尔边界的喜马拉雅山脉。  \n   - 珠穆朗玛峰是地球上最高的山峰，也是登山者最向往的目标之一。\n\n2. **乔戈里峰（K2）**  \n   - **海拔高度**：8,611米（28,251英尺）  \n   - **位置**：位于巴基斯坦与中国交界的喀喇昆仑山脉。  \n   - K2被称为“野蛮之山”，因其攀登难度极高，仅次于珠穆朗玛峰的高度使其成为世界第二高峰。\n\n3. **干城章嘉峰（Kangchenjunga）**  \n   - **海拔高度**：8,586米（28,169英尺）  \n   - **位置**：位于尼泊尔和印度锡金邦交界的喜马拉雅山脉。  \n   - 干城章嘉峰是世界第三高峰，同时也是印度的最高峰。\n\n这些山峰都位于亚洲的喜马拉雅山脉或喀喇昆仑山脉，是地理和自然的奇观。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 328, 'prompt_tokens': 15, 'total_tokens': 343, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-a663c01b-53c0-4

也可進一步的使用 OpenAI chat completions 的 JSON messages 格式引導語言模型的回應，這樣可以更精準的控制語言模型的回應。

In [105]:
chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)

messages = [
    SystemMessage(content="你是用正體中文的 AI 助手，簡潔回覆"),
    HumanMessage(content="世界上最高的三座山?"),
]

respone = chat.invoke(messages)
print(respone)

content='1. 聖母峰（珠穆朗瑪峰） - 8848.86 公尺  \n2. 喬戈里峰（K2） - 8611 公尺  \n3. 干城章嘉峰 - 8586 公尺  ' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 35, 'total_tokens': 92, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-c4b3ff89-3fc2-4ec4-ba4a-b10ce302a184-0' usage_metadata={'input_tokens': 35, 'output_tokens': 57, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


由語言模型回傳的內容，可以透過各種 Parser 簡化處理解析與輸出的程式碼的複雜度，以下是一個簡單的範例，將前述對話回應以 StrOutputParser 來解析回傳的字串內容。

In [106]:
from langchain_core.output_parsers import StrOutputParser

chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)

messages = [
    SystemMessage(content="你是用正體中文的 AI 助手，簡潔回覆"),
    HumanMessage(content="世界上最高的三座山?"),
]

respone = chat.invoke(messages)
print(respone)

parser = StrOutputParser()
print (parser.invoke(respone)) 
print(type(parser.invoke(respone)))

content='1. 聖母峰（珠穆朗瑪峰）- 8,848.86公尺  \n2. 喬戈里峰（K2）- 8,611公尺  \n3. 干城章嘉峰 - 8,586公尺  ' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 35, 'total_tokens': 95, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-7b2ef526-ee92-4724-a56a-cab7574e8889-0' usage_metadata={'input_tokens': 35, 'output_tokens': 60, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
1. 聖母峰（珠穆朗瑪峰）- 8,848.86公尺  
2. 喬戈里峰（K2）- 8,611公尺  
3. 干城章嘉峰 - 8,586公尺  
<class 'str'>


將前述對話回應以 CommaSeparatedListOutputParser 來解析回傳的字串，產生 List 結構之回覆內容。

In [107]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser 

chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)

messages = [
    SystemMessage(content="你是用正體中文的 AI 助手，簡潔回覆"),
    HumanMessage(content="世界上最高的三座山?"),
]

respone = chat.invoke(messages)
print(respone)

parser = CommaSeparatedListOutputParser()
print (parser.invoke(respone)) 
print(type(parser.invoke(respone)))

content='1. 聖母峰（珠穆朗瑪峰）：8,848.86 公尺  \n2. 喬戈里峰（K2）：8,611 公尺  \n3. 干城章嘉峰：8,586 公尺' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 35, 'total_tokens': 89, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-096bcb7a-b990-49d0-8001-9408a18b3b24-0' usage_metadata={'input_tokens': 35, 'output_tokens': 54, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
['1. 聖母峰（珠穆朗瑪峰）：8', '848.86 公尺  \n2. 喬戈里峰（K2）：8', '611 公尺  \n3. 干城章嘉峰：8', '586 公尺']
<class 'list'>


接下嘗試使用 Chain 來處理相同問題，LLMChain 是最基本的 Chain，能夠以 PromptTemplate 增加輸入提示時的彈性，並將中間處理的大型語言模型抽象化，最後視狀況可以搭配多種 OutputParser 來處理輸出。由接下來的範例可以看出，我們可將 LLM Model 的 AzureOpenAI 與 Chat Model 的 AzureChatOpenAI 都使用一致的 LLMChain 鍊來進行處理。

In [108]:
chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)
# 設定 PromptTemplate
prompt = PromptTemplate(input_variables=["prompt_str"],template="{prompt_str}")

# 將 AzureOpenAI 以 LLMChain 方式使用

chain = prompt | chat
response = chain.invoke({"prompt_str":"世界上最高的兩座山?"})

# 單純輸出內容是字串
print(response)
print(type(response))

content='世界上最高的兩座山是：\n\n1. **珠穆朗玛峰（Mount Everest）**  \n   - 海拔：8,848.86米  \n   - 所在地：位于中国和尼泊尔边界的喜马拉雅山脉。  \n   - 珠穆朗玛峰是地球上海拔最高的山峰，被认为是“世界之巅”。\n\n2. **乔戈里峰（K2）**  \n   - 海拔：8,611米  \n   - 所在地：位于中国和巴基斯坦边界的喀喇昆仑山脉。  \n   - 乔戈里峰是世界第二高峰，以其陡峭和极具挑战性的攀登条件闻名，被称为“野蛮之山”。\n\n这两座山峰都属于亚洲的高山体系，是世界上最著名的山峰之一，也是许多登山者梦寐以求的挑战目标。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 216, 'prompt_tokens': 15, 'total_tokens': 231, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-ea0050c3-42d9-41f8-aacf-7c455cbb5d11-0' usage_metadata={'input_tokens': 15, 'output_tokens': 216, 'total_tokens': 231, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

LangChain 也提供針對對話形式之 ChatPromptTemplate 來進行對話的輸入，這樣可以更方便的進行對話的處理。

In [109]:
from langchain_core.prompts import ChatPromptTemplate

chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)

# 設定 ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("system", "你是名叫 {name}，一個用精煉文言文回覆正體中文的 AI 助手"),
    ("human", "{user_input}"),
])

chain = prompt | chat
response = chain.invoke({"name": "Tom","user_input": "你是誰?世界上最高的三座山?"})
print(response)

content='吾乃名喚 Tom 之智械助手，專應汝之疑問。\n\n夫天下巍峨之巔，以高聳而聞名者，首推三座如下：  \n一、**珠穆朗瑪峰**，高八千八百四十八公尺，位於喜馬拉雅山脈中，乃全球之最。  \n二、**喬戈里峰**（K2），高八千六百十一公尺，居次，坐落於喀喇崑崙山脈。  \n三、**干城章嘉峰**（Kangchenjunga），高八千五百八十六公尺，位於尼泊爾與印度之交界，列第三。  \n\n三山之巔，直插霄漢，令人景仰。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 185, 'prompt_tokens': 47, 'total_tokens': 232, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-21019004-2ef3-4f7a-9d75-c349e0c26ca9-0' usage_metadata={'input_tokens': 47, 'output_tokens': 185, 'total_tokens': 232, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


將前述所有用到的元件組合成一個完整的 Chain，並執行這個 Chain 來完成對話。

In [110]:
chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)

# 設定 ChatPromptTemplate
prompt = ChatPromptTemplate([
    ("system", "你是名叫 {name}，一個用精煉文言文回覆正體中文的 AI 助手"),
    ("human", "{user_input}"),
])

# 設定 Output Parser 為 StrOutputParser 
parser = StrOutputParser ()

# 鏈接 PromptTemplate、Chat Model、OutputParser
chain = prompt | chat | parser

# 執行鏈並顯示回應
response = chain.invoke({"name": "Tom","user_input": "你是誰?世界上最高的三座山?"})
print(response)

吾乃湯姆，爾之忠僕也。  
夫天下之巔，三高者，分列如下：  
其一，珠穆朗瑪峰，高八千八百四十八公尺，世界之最。  
其二，喬戈里峰（K2），高八千六百十一公尺，次之。  
其三，干城章嘉峰（Kangchenjunga），高八千五百八十六公尺，居第三。  
此三峰，皆立於喜馬拉雅山脈，巍然聳立，壯哉！


## 3. 使用文件載入工具載入 Wikipedia 內容

LangChain 提供豐富的文件載入 (Documenet Loader) 工具，以便將一般文字檔， CSV 格式檔案，PDF 格式檔案，JSON 格式檔案等眾多檔案格式以一致的 Document 物件來被框架內其他物件所使用，目前 LangChain 有超過七十種以上的文件載入工具可以供選用。接下來以能夠搜尋與載入 Wikipedia 內容 的 WikipediaLoader 作為範例。

In [111]:
from langchain_community.document_loaders import WikipediaLoader

# 查詢 Wikipedia 台灣蘭嶼條目，透過 load_max_docs 參數限制最多只載入兩篇內容
docs = WikipediaLoader(query="蘭嶼", load_max_docs=2).load()
len(docs)

2

In [112]:
# 顯示第一篇內容的 Metadata
print (docs[0].metadata)

{'title': 'Orchid Island', 'summary': "Orchid Island, also known by other names, is a 45 km2 (17 sq mi) volcanic island off the southeastern coast of Taiwan, which Orchid Island is part of. It is separated from the Batanes of the Philippines by the Bashi Channel of the Luzon Strait. The island and the nearby Lesser Orchid Island are governed as Lanyu Township in Taitung County, which is one of the county's two insular townships (the other being Lyudao Township).\nIt is considered a potential World Heritage Site.", 'source': 'https://en.wikipedia.org/wiki/Orchid_Island'}


In [113]:
# 顯示第一篇內容前 200 字
content = docs[0].page_content[:200]  
print(content)

Orchid Island, also known by other names, is a 45 km2 (17 sq mi) volcanic island off the southeastern coast of Taiwan, which Orchid Island is part of. It is separated from the Batanes of the Philippin


## 4 使用載入的文件詢問問題

以基礎模型來回答問題

In [114]:
# 完全依賴基礎模型回答問題，很高的機會得到來自幻覺的答案
chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver,temperature=0.5)
respone = chat.invoke("只依據事實回答，蘭嶼鄉鄉長姓名?")
print(respone)

content='截至我的知識截止日期2023年10月，蘭嶼鄉鄉長是**夏曼·迦拉牧**。如果您需要最新資訊，建議查詢蘭嶼鄉公所的官方網站或相關政府公告。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 26, 'total_tokens': 86, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-b559b5b6-1aa7-4cc9-99d2-d88c20ecf747-0' usage_metadata={'input_tokens': 26, 'output_tokens': 60, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


使用之前已經建立的 chat 來詢問 Wikipedia 下載的文件內容，由回覆的答案可以知道並非來自基礎模型，而是來自下載的 Wikipedia 文件

In [115]:
# 使用 Wikipeida 載入之文件內容回答問題，無法回答此問題
prompt = "只依據以下事實回答，\n事實:"+content+ "，\n蘭嶼鄉鄉長姓名?"
respone = chat.invoke(prompt)
print(respone)

content='抱歉，根據您提供的事實中，並未提及蘭嶼鄉鄉長的姓名，因此無法回答此問題。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 80, 'total_tokens': 115, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-e7f64c83-5166-4adf-9004-6902280e9279-0' usage_metadata={'input_tokens': 80, 'output_tokens': 35, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


也可以利用 prompt template 來增加提示的彈性，只是需要將帶入前面取得之 Wikipedia 內容代入 PromptTemplate 物件輸出轉換為字串

In [116]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["doc_content"], template="只依據以下事實回答，\n事實:{doc_content}\n蘭嶼鄉鄉長姓名?")
prompt_formatted_str: str = prompt_template.format(doc_content = content)

respone = chat.invoke(prompt_formatted_str)
print(respone)


content='根據您提供的事實中，並未提及蘭嶼鄉鄉長的姓名，因此無法回答該問題。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 80, 'total_tokens': 113, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}} id='run-b2aac70a-caea-4686-a813-916b347e8c81-0' usage_metadata={'input_tokens': 80, 'output_tokens': 33, 'total_tokens': 113, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


## 5 練習在 LangChain 內使用 Azure OpenAI Embeddings
利用台北市政府開放資料 [112年度美食在台北「鍋際大賞」店家名單](https://data.taipei/dataset/detail?id=7eec2e8f-02f9-4100-9171-66f6667b8b3d) 為範例，以 LangChain 所提供的 CSV 載入工具，將資料載入後，再使用 Azure OpenAI Embeddings 進行店家名稱與地址的向量計算。並以 Meta 公司 Facebook AI 團隊開放原始程式碼向量資料庫 FAISS 將向量值儲存在本機的檔案系統中，以便後續的向量近似搜尋，FAISS 的近似比對是採用 L2 norm 方式來將兩個向量間的距離正規化，分數越接近零代表兩個向量越相似，這與 OpenAI Embeddings 的餘弦近似 (cosine similarity) 正規化後的分數值不一樣，餘弦近似分數越接近 1 的則兩個向量越相似。

In [117]:
from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS

# 建立 Azure OpenAI Embeddings 類別的實例
embeddings_model = AzureOpenAIEmbeddings(
    model=emb_model,
    azure_endpoint=azure_openai_endpoint,
    openai_api_version=api_ver,
    chunk_size = 16
)

# 以 LangChain 的 CSVLoader 類別，讀取 CSV 檔案並載入文件
loader = CSVLoader(file_path="./data/2023-taipei-hot pot-restaurant.csv",
                   csv_args={
                    "delimiter": ",",                    
                    "fieldnames": ["序號","區域","店家","報名組別","營業地址","Longitude","Latitude"],
                    },                    
                   encoding="utf-8")
docs = loader.load()

# 顯示 docs 內的第一筆資料
print (docs[1].page_content)

# 載入 CSV 文件計算 OpenAI Embeddings 向量值，並建立 FAISS 向量資料庫索引
db = FAISS.from_documents(documents=docs, embedding=embeddings_model)

# 將 FAISS 向量資料索引儲存於本機磁碟 
db.save_local("./data/FAISS/faiss_index")

序號: 1
區域: 中山區
店家: 滿堂紅頂級麻辣鴛鴦鍋
報名組別: 經典鍋物組(799以下)
營業地址: 臺北市中山區松江路185號2樓
Longitude: 121.5331715
Latitude: 25.05724095


以下重新載入 FAISS 向量值索引，並開始利用 FAISS 進行向量近似比對

In [118]:
from langchain_openai.embeddings.azure import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS

# 建立 Azure OpenAI Embeddings 類別的實例
embeddings_model = AzureOpenAIEmbeddings(
    model=emb_model,
    azure_endpoint=azure_openai_endpoint,
    openai_api_version=api_ver,
    chunk_size = 16
)

# 如果資料沒有變更未來僅需 db = FAISS.load_local("faiss_index", embeddings_model) 方式即可載入之前建立的向量索引，無須重新計算文件的向量值
db = FAISS.load_local("./data/FAISS/faiss_index", embeddings_model,allow_dangerous_deserialization=True)

In [119]:
# 直接傳入查詢字串方式來進行相似度搜尋
query = "士林"
docs_and_scores = db.similarity_search_with_score(query)

# 顯示 docs_and_scores 全部內容
for doc in docs_and_scores:
    print(doc)  


(Document(metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 60}, page_content='序號: 60\n區域: 士林區\n店家: 錢都日式涮涮鍋-士林芝山店\n報名組別: 經典鍋物組(799以下)\n營業地址: 臺北市士林區福國路71號-1\nLongitude: 121.5232239\nLatitude: 25.10250454'), 0.40565524)
(Document(metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 61}, page_content='序號: 61\n區域: 士林區\n店家: 藏王極上鍋物\n報名組別: 饗宴鍋物組(800以上)\n營業地址: 臺北市士林區忠誠路二段55號3樓\nLongitude: 121.5310986\nLatitude: 25.11282254'), 0.40968293)
(Document(metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 59}, page_content='序號: 59\n區域: 士林區\n店家: 川邸鍋物專門\n報名組別: 經典鍋物組(799以下)\n營業地址: 臺北市士林區承德路四段79號1樓\nLongitude: 121.5232675\nLatitude: 25.08314465'), 0.41156936)
(Document(metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 52}, page_content='序號: 52\n區域: 信義區\n店家: 王鍋屋Shabu ong 京都風味酸白菜鍋專門店\n報名組別: 饗宴鍋物組(800以上)\n營業地址: 臺北市信義區逸仙路50巷20號1樓\nLongitude: 121.5626957\nLatitude: 25.04119888'), 0.44357985)


In [120]:
# 以計算查詢字串向量方式來進行相似度搜尋
query = "滿堂紅"
embedding_vector = embeddings_model.embed_query(query)
docs_and_scores = db.similarity_search_by_vector(embedding_vector)

# 顯示 docs_and_scores 全部內容
for doc in docs_and_scores:
    print(doc)  

page_content='序號: 1
區域: 中山區
店家: 滿堂紅頂級麻辣鴛鴦鍋
報名組別: 經典鍋物組(799以下)
營業地址: 臺北市中山區松江路185號2樓
Longitude: 121.5331715
Latitude: 25.05724095' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 1}
page_content='序號: 30
區域: 松山區
店家: 芳朵麻辣鍋 Fondue M spicy pot
報名組別: 饗宴鍋物組(800以上)
營業地址: 臺北市松山區光復北路98號2樓
Longitude: 121.5574816
Latitude: 25.05121754' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 30}
page_content='序號: 23
區域: 大同區
店家: 本鼎堂台式漢方麻辣鍋
報名組別: 饗宴鍋物組(800以上)
營業地址: 臺北市大同區南京西路277號1樓
Longitude: 121.5094827
Latitude: 25.05408046' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 23}
page_content='序號: 39
區域: 大安區
店家: 太和殿
報名組別: 饗宴鍋物組(800以上)
營業地址: 臺北市大安區信義路四段315號1樓
Longitude: 121.5560896
Latitude: 25.03404847' metadata={'source': './data/2023-taipei-hot pot-restaurant.csv', 'row': 39}


## 6 在 LangChain 內使用工具 (Tool) 自訂延伸功能
大型語言模型的快速進步，與人類直接互動能夠處理的工作越來越多，然而大型模型對數值運算類型工作無法確保計算正確，此外如果我們希望應用系統能夠直接與資料庫或現有成熟的 API 互動，來增添大型語言模型能力的不足之處，此時 LangChain 的[工具 (Tool)](https://python.langchain.com/docs/concepts/tool_calling/) 機制；可讓軟體開發人員可以透過適當的工具自訂各種功能，並結合大型語言模型自動選擇呼叫正確的工具並找出正確的參數傳遞給工具，以增添應用系統的能力。

以下程式碼示範如何建立一個工具:

In [25]:
from langchain_core.tools import tool

# 建立一個名為 squarekm_to_ping 的工具，可將面積平方公里轉換為台灣常用的坪數
@tool
def squarekm_to_ping(squarekm:int) -> str:
    """the squarekm_to_ping can convert square kilometers to pings
    Args:
        squarekm (int): square kilometers as input
    Returns:
        str: pings as output    
    """

    return str(squarekm * 302500)

# 建立一個名為 squarekm_to_acres 的工具，可將面積平方公里轉換為英畝
@tool
def squarekm_to_acres(squarekm:int) -> str:
    """the squarekm_to_acres can convert square kilometers to acres
    Args:
        squarekm (int): square kilometers as input
    Returns:
        str: acres as output    
    """

    return str(squarekm * 247.105)

以下程式碼示範如何綁定一個工具或多個工具到 LangChain 所建立的大型語言模型上。

In [ ]:
# 由於 tool 實作了 runnable 介面 ，可以直接呼叫 invoke 以測試工具呼叫結果
result = squarekm_to_ping.invoke({"squarekm": 45})
print (result)

# tools 定義可以使用使用哪些工具
tools = [squarekm_to_ping,squarekm_to_acres]

# 使用 Azure OpenAI Service 建立一個 chat 模型，並將工具綁定 (Tool binding) 此 chat 模型 
chat = AzureChatOpenAI(azure_deployment=chat_model,api_version=api_ver)
model_with_tools = chat.bind_tools(tools)

# 呼叫工具 
response = model_with_tools.invoke("蘭嶼面積有多少英畝?")
# 顯示回應內容
print(response)
# 顯示回應中針對 tool_calls 的解析結果
print(response.tool_calls)
# 顯示應該呼叫的 Tool 名稱
print(response.tool_calls[0]["name"])
# 顯示應該呼叫的 Tool 參數
print(response.tool_calls[0]["args"])
# 依據回應結果選擇對應之工具，並傳入正確之參數得到最終結果
selected_tool = {"squarekm_to_ping": squarekm_to_ping, "squarekm_to_acres": squarekm_to_acres}[response.tool_calls[0]["name"].lower()]
tool_msg = selected_tool.invoke(response.tool_calls[0]["args"])
print(tool_msg)


13612500
content='' additional_kwargs={'tool_calls': [{'id': 'call_hEFN2mA694u9i89W7YvIrSY7', 'function': {'arguments': '{"squarekm":45}', 'name': 'squarekm_to_acres'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 142, 'total_tokens': 160, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b705f0c291', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_res